In [82]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlalchemy
from keys import MySQL_Pass
from keys import OpenWeather_API_key

In [83]:
schema = "sql_population"
host = "127.0.0.1"
user = "root"
password = MySQL_Pass
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

cities_df = pd.read_sql('SELECT city_id, city FROM cities', connection_string)

In [84]:
cities_df['city'][2]

'Munich'

In [85]:
def get_weather_info(cities_df):
    API_key = OpenWeather_API_key
    city_name = []
    forecast_time = []
    rain_in_last_3h = []
    outlook = []
    temp = []
    feels_like = []
    rain_prob = []
    wind_speed = []
    city_id = []
    
    for index, row in cities_df.iterrows():
        city = row['city']
        weather_data = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
        weather_data_json = weather_data.json()
        
        #print(weather_data_json)  # Print the JSON response
        
        weather_list = weather_data_json.get("list", [])  # Use .get() to handle missing keys
        
        for i, dt in enumerate(weather_list):
            forecast_time.append(weather_list[i]["dt"])
            temp.append(weather_list[i]['main']["temp"])
            feels_like.append(weather_list[i]['main']["feels_like"])
            rain_prob.append(weather_list[i]['pop'])
            #rain_in_last_3h.append(weather_list[i]['rain']['3h'])
            wind_speed.append(weather_list[i]['wind']['speed'])
            outlook.append(weather_list[i]['weather'][0]["description"])
            city_name.append(city)
            city_id.append(cities_df['city_id'][index])
            
    weather_df = pd.DataFrame(list(zip(city_id, forecast_time, outlook,temp,  feels_like, wind_speed , rain_prob)), 
                                  columns=["city_id", "forecast_time", "outlook", "temperature",  "feels_like", "wind_speed", "rain_prob"])
    weather_df['forecast_time'] = pd.to_datetime(weather_df['forecast_time'], unit='s')
        
    return weather_df

In [86]:
weather_df = get_weather_info(cities_df)

In [87]:
weather_df

,city_id,forecast_time,outlook,temperature,feels_like,wind_speed,rain_prob
0,1,2024-02-13 15:00:00,broken clouds,9.08,6.86,4.03,0.0
1,1,2024-02-13 18:00:00,broken clouds,7.84,5.87,3.07,0.0
2,1,2024-02-13 21:00:00,broken clouds,5.97,4.06,2.49,0.0
3,1,2024-02-14 00:00:00,overcast clouds,3.92,1.80,2.31,0.0
4,1,2024-02-14 03:00:00,overcast clouds,3.44,0.77,2.83,0.0
...,...,...,...,...,...,...,...
115,3,2024-02-18 00:00:00,light rain,6.76,6.07,1.41,0.2
116,3,2024-02-18 03:00:00,overcast clouds,6.36,5.49,1.51,0.0
117,3,2024-02-18 06:00:00,broken clouds,4.34,2.95,1.69,0.0
118,3,2024-02-18 09:00:00,scattered clouds,6.36,5.12,1.83,0.0


In [78]:
weather_df.to_sql('weather',
                  if_exists='append',
                  con=connection_string,
                  index=False)

120